# Formation South Green 2021  

##  Initiation à l’analyse de données Minion

### PART 3

Created by J.Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-CIRAD)

Septembre 2021

You can found assemblies results files for the whole of Clones in the DATA/AGGREGATED repertory.

# 1. Perform a comparison of assemblies using QUAST

The metrics of assemblies can be evaluated using a quality assessment tools such as [QUAST](http://quast.bioinf.spbau.ru/manual.html).

In part 2, we have assembled raw data using various assemblers and also polished and corrected it. 

Aggregate all assemblies you have produced for your favorite clone in a new folder named "AGGREGATED".

We're going to compare it with QUAST.

In [3]:
# active quast env
conda activate quast

(quast) 

: 1

In [18]:
CLONE=Clone20
cd ~/SG-ONT-2021/RESULTS
mkdir -p ~/SG-ONT-2021/RESULTS/AGGREGATED
cd ~/SG-ONT-2021/RESULTS/AGGREGATED
ln -s ~/SG-ONT-2021/RESULTS/FLYE/assembly.fasta ${CLONE}_FLYE.fasta
ln -s ~/SG-ONT-2021/RESULTS/FLYE_RACON/assembly.racon2.fasta ${CLONE}_FLYE_RACONx2.fasta
ln -s ~/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta ${CLONE}_FLYE_RACONx2_MEDAKA.fasta
ls -l ~/SG-ONT-2021/RESULTS/AGGREGATED

(quast) (quast) (quast) (quast) (quast) (quast) (quast) total 0
lrwxrwxrwx 1 jovyan users 52 Sep 15 09:48 Clone20_FLYE.fasta -> /home/jovyan/SG-ONT-2021/RESULTS/FLYE/assembly.fasta
lrwxrwxrwx 1 jovyan users 65 Sep 15 09:48 Clone20_FLYE_RACONx2.fasta -> /home/jovyan/SG-ONT-2021/RESULTS/FLYE_RACON/assembly.racon2.fasta
lrwxrwxrwx 1 jovyan users 83 Sep 15 09:48 Clone20_FLYE_RACONx2_MEDAKA.fasta -> /home/jovyan/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta
(quast) 

: 1

#### Do similar symbolic links to RAVEN results and check your forder

## 1.1  Run QUAST on CLONE assemblies and compare them

In [19]:
cd ~/SG-ONT-2021/RESULTS/AGGREGATED/
quast.py *.fasta -o QUAST

(quast) /opt/conda/envs/quast/bin/quast.py Clone20_FLYE.fasta Clone20_FLYE_RACONx2.fasta Clone20_FLYE_RACONx2_MEDAKA.fasta -o QUAST

Version: 5.0.2

System information:
  OS: Linux-4.19.0-16-cloud-amd64-x86_64-with-debian-bullseye-sid (linux_64)
  Python version: 3.7.10
  CPUs number: 4

Started: 2021-09-15 09:48:48

Logging to /home/jovyan/SG-ONT-2021/RESULTS/AGGREGATED/QUAST/quast.log
NOTICE: Maximum number of threads is set to 1 (use --threads option to set it manually)

CWD: /home/jovyan/SG-ONT-2021/RESULTS/AGGREGATED
Main parameters: 
  MODE: default, threads: 1, minimum contig length: 500, minimum alignment length: 65, \
  ambiguity: one, threshold for extensive misassembly size: 1000

Contigs:
  Pre-processing...
  1  Clone20_FLYE.fasta ==> Clone20_FLYE
  2  Clone20_FLYE_RACONx2.fasta ==> Clone20_FLYE_RACONx2
  3  Clone20_FLYE_RACONx2_MEDAKA.fasta ==> Clone20_FLYE_RACONx2_MEDAKA

2021-09-15 09:48:48
Running Basic statistics processor...
  Contig files: 
    1  Clone20_FLYE
    2

: 1

#### Go to the QUAST directory and check the file content.

#### Looking at the output statistics, what are the main difference between the assemblies?

#### Look total sizes, number of contigs, N50 statistics.

#### For your favorite clone what is the best one ?

## 1.2  Comparison of the assemblies against the reference genome of closely organism (REAL DATA !)

It's possible to compare assemblies against the reference genome of closely related organism. 

In our case we will use the real dataset Hh. 

Previously calculated assemblies for Hh can be found in DATA/real_Hh/Hh-ASSEMBLIES

In [1]:
ls -lh ~/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES

total 763M
-rw-r----- 1 jovyan users 2.6M Sep 13 09:14 CANU-STEP_ASSEMBLY-CIRCULARISED.fasta
-rw-r----- 1 jovyan users 1.1K Sep 13 09:14 CANU-STEP_ASSEMBLY-CIRCULARISED.fasta.fai
-rw-r----- 1 jovyan users 8.7M Sep 13 09:14 CANU-STEP_ASSEMBLY-CIRCULARISED.fasta.mmi
-rw-r----- 1 jovyan users 2.6M Sep 13 09:14 CANU-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta
-rw-r----- 1 jovyan users 1.1K Sep 13 09:14 CANU-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta.fai
-rw-r----- 1 jovyan users 8.7M Sep 13 09:14 CANU-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta.mmi
-rw-r----- 1 jovyan users 2.6M Sep 13 09:14 CANU-STEP_POLISHING_RACON-CIRCULARISED.fasta
-rw-r----- 1 jovyan users 1.3K Sep 13 09:14 CANU-STEP_POLISHING_RACON-CIRCULARISED.fasta.fai
-rw-r----- 1 jovyan users 8.7M Sep 13 09:14 CANU-STEP_POLISHING_RACON-CIRCULARISED.fasta.mmi
-rw-r----- 1 jovyan users 2.2M Sep 13 09:14 FLYE-STEP_ASSEMBLY-CIRCULARISED.fasta
-rw-r----- 1 jovyan users  200 Sep 13 09:14 FLYE-STEP_ASSEMBLY-C

### Compare Hh assemblies against the reference sequence using QUAST

#### WARNING : This can take a while!! 

In [22]:
mkdir ~/SG-ONT-2021/RESULTS/AGGREGATED_Hh/
cd ~/SG-ONT-2021/RESULTS/AGGREGATED_Hh/
time quast.py /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/*.fasta -R /home/jovyan/SG-ONT-2021/DATA/real_Hh/REFH_M1C132.fasta -o QUAST_REF

mkdir: cannot create directory ‘/home/jovyan/SG-ONT-2021/RESULTS/AGGREGATED_Hh/’: File exists
(quast) (quast) (quast) /opt/conda/envs/quast/bin/quast.py /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/CANU-STEP_ASSEMBLY-CIRCULARISED.fasta /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/CANU-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/CANU-STEP_POLISHING_RACON-CIRCULARISED.fasta /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/FLYE-STEP_ASSEMBLY-CIRCULARISED.fasta /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/FLYE-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/FLYE-STEP_POLISHING_RACON-CIRCULARISED.fasta /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/MINIASM-STEP_ASSEMBLY-CIRCULARISED.fasta /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/MINIASM-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMB

#### Which of the assemblies are closest in size relative to the reference ?

#### Does this have the largest N50 and fewest number of contigs?

The ideal shape of the Nx graph would be vertical line if the bacteria is a single contig. 

As more contigs are present, the cumulative length graph shifts to the right slightly as contigs get shorter.

#### Does the GC content graph indicate contamination ?

#### What assembly does obtain the best contiguity metrics?


# 2. Assessing gene space using BUSCO (REAL DATA)

Benchmarking Universal Single-Copy Orthologs ([BUSCO](https://busco.ezlab.org/busco_userguide.html)) helps to check if you have a good assembly, by searching the expected single-copy lineage-conserved orthologs in any newly-sequenced genome from an appropriate phylogenetic clade.

Calculate the gene space look like for each assembly?

In [3]:
mkdir -p ~/SG-ONT-2021/RESULTS/AGGREGATED_Hh/BUSCO
cd ~/SG-ONT-2021/RESULTS/AGGREGATED_Hh/BUSCO

(busco) (busco) 

: 1

Lineage can be choose in busco database using --luist-datasets parameter

In [6]:
# busco env
conda activate busco
busco --list-datasets

(busco) INFO:	Downloading information on latest versions of BUSCO data...
INFO:	Downloading file 'https://busco-data.ezlab.org/v5/data/information/lineages_list.2019-11-27.txt.tar.gz'
INFO:	Decompressing file '/home/jovyan/SG-ONT-2021/RESULTS/AGGREGATED_Hh/BUSCO/busco_downloads/information/lineages_list.2019-11-27.txt.tar.gz'

################################################

Datasets available to be used with BUSCOv4 as of 2019/11/27:

 bacteria_odb10
     - acidobacteria_odb10
     - actinobacteria_phylum_odb10
         - actinobacteria_class_odb10
             - corynebacteriales_odb10
             - micrococcales_odb10
             - propionibacteriales_odb10
             - streptomycetales_odb10
             - streptosporangiales_odb10
         - coriobacteriia_odb10
             - coriobacteriales_odb10
     - aquificae_odb10
     - bacteroidetes-chlorobi_group_odb10
         - bacteroidetes_odb10
             - bacteroidia_odb10
                 - bacteroidales_odb10
           

: 1

In [7]:
ASSEMBLY="~/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/FLYE-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta"
LINEAGE=bacteria_odb10
busco -i "$ASSEMBLY" -l "$LINEAGE" -c "${CPUS:-4}" -m genome -o BUSCO_RESULTS

(busco) (busco) INFO:	***** Start a BUSCO v5.1.3 analysis, current time: 09/17/2021 09:11:42 *****
INFO:	Configuring BUSCO with local environment
INFO:	Mode is genome
INFO:	Downloading information on latest versions of BUSCO data...
INFO:	Input file is /home/jovyan/SG-ONT-2021/DATA/real_Hh/Hh-ASSEMBLIES/FLYE-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta
INFO:	Downloading file 'https://busco-data.ezlab.org/v5/data/lineages/bacteria_odb10.2020-03-06.tar.gz'
INFO:	Decompressing file '/home/jovyan/SG-ONT-2021/RESULTS/AGGREGATED_Hh/BUSCO/busco_downloads/lineages/bacteria_odb10.tar.gz'
INFO:	Running BUSCO using lineage dataset bacteria_odb10 (prokaryota, 2020-03-06)
INFO:	***** Run Prodigal on input to predict and extract genes *****
INFO:	Running Prodigal with genetic code 11 in single mode
INFO:	Running 1 job(s) on prodigal, starting at 09/17/2021 09:11:43
INFO:	[prodigal]	1 of 1 task(s) completed
INFO:	Genetic code 11 selected as optimal
INFO:	***** Run HMMER on gene sequences ****

: 1

In [8]:
conda deactivate

### What does the gene space look like for this assembly?

#### Run BUSCO in all assemblies generated if you want... 

# 3. Read alignment statistics - remapping (CLONE)

Read congruency is an important measure in determining assembly accuracy. Clusters of read pairs that align incorrectly are strong indicators of mis-assembly.

How well do the reads align back to the draft assemblies? Use minimap2 and samtools to assess the basic alignment statistics.

Make a folder for your results.

We will use CloneX_FLYE_RACONx2_MEDAKA.fasta assembly and ONT READS as well ILLUMINA READS.

In [34]:
mkdir -p ~/SG-ONT-2021/RESULTS/REMAPPING
cd ~/SG-ONT-2021/RESULTS/REMAPPING/

In [35]:
CLONE="Clone20"
ASSEMBLY="/home/jovyan/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta"
ONT="/home/jovyan/SG-ONT-2021/DATA/Clone20/ONT/Clone20.fastq.gz"
ILLUMINA_R1="/home/jovyan/SG-ONT-2021/DATA/${CLONE}/ILL/${CLONE}_R1.fastq.gz"
ILLUMINA_R2="/home/jovyan/SG-ONT-2021/DATA/${CLONE}/ILL/${CLONE}_R2.fastq.gz"

In [36]:
#symbolic link from last assembly in the current repertory ReMAPPING
ln -s ${ASSEMBLY} ${CLONE}_FLYE_RACONx2_MEDAKA.fasta
# overwrite ASSEMBLY variable
ASSEMBLY=${CLONE}_FLYE_RACONx2_MEDAKA.fasta

## Mapping assemblies vs ONT reads

In [37]:
minimap2 -ax map-ont -t 4 ${ASSEMBLY} ${ONT} | samtools sort -@ 1 -T "${ASSEMBLY/.fasta/}" -O BAM -o "${ASSEMBLY/.fasta/_ONT_minimap2.bam}" -
samtools index "${ASSEMBLY/.fasta/_ONT_minimap2.bam}"
samtools view -F 0x904 -c "${ASSEMBLY/.fasta/_ONT_minimap2.bam}"

[M::mm_idx_gen::0.080*1.04] collected minimizers
[M::mm_idx_gen::0.116*1.42] sorted minimizers
[M::main::0.116*1.42] loaded/built the index for 3 target sequence(s)
[M::mm_mapopt_update::0.126*1.38] mid_occ = 6
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 3
[M::mm_idx_stat::0.132*1.36] distinct minimizers: 214382 (97.69% are singletons); average occurrences: 1.032; average spacing: 5.355
[M::worker_pipeline::23.216*2.64] mapped 10804 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -ax map-ont -t 4 Clone20_FLYE_RACONx2_MEDAKA.fasta /home/jovyan/SG-ONT-2021/DATA/Clone20/ONT/Clone20.fastq.gz
[M::main] Real time: 23.229 sec; CPU: 61.394 sec; Peak RSS: 0.392 GB
10804


## Mapping assemblies vs ILLUMINA reads

In [38]:
#minimap2 -ax sr ref.fa read1.fq read2.fq > aln.sam
minimap2 -ax sr -t 4 ${ASSEMBLY} ${ILLUMINA_R1} ${ILLUMINA_R2} | samtools sort -@ 1 -T "${ASSEMBLY/.fasta/}" -O BAM -o "${ASSEMBLY/.fasta/_ILL_minimap2.bam}" -
samtools index "${ASSEMBLY/.fasta/_ILL_minimap2.bam}"
samtools view -F 0x904 -c "${ASSEMBLY/.fasta/_ILL_minimap2.bam}"

[M::mm_idx_gen::0.077*1.03] collected minimizers
[M::mm_idx_gen::0.117*1.51] sorted minimizers
[M::main::0.117*1.51] loaded/built the index for 3 target sequence(s)
[M::mm_mapopt_update::0.117*1.51] mid_occ = 1000
[M::mm_idx_stat] kmer size: 21; skip: 11; is_hpc: 0; #seq: 3
[M::mm_idx_stat::0.128*1.41] distinct minimizers: 194234 (98.68% are singletons); average occurrences: 1.018; average spacing: 5.991
[M::worker_pipeline::6.584*2.10] mapped 290400 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -ax sr -t 4 Clone20_FLYE_RACONx2_MEDAKA.fasta /home/jovyan/SG-ONT-2021/DATA/Clone20/ILL/Clone20_R1.fastq.gz /home/jovyan/SG-ONT-2021/DATA/Clone20/ILL/Clone20_R2.fastq.gz
[M::main] Real time: 6.603 sec; CPU: 13.857 sec; Peak RSS: 0.176 GB
290400


### What is the percent of aligned ONT and ILLUMINa reads in your clone assembly?

# 4. Blobtools

During the sequence quality assessment stage we tried to discern whether contamination was present. Sometimes this is not feasible at the read level. By plotting Contig GC content vs Contig Read Coverage we can look for clusters of contigs that share similar coverage. The appearance of multiple clusters can indicate multiple organisms. Occasionally, contigs can also be taxonomically classified, providing further evidence for contaminants.

Run Blobtools on each assembly. Blobtools requires both a BAM file as input and blast output for the classification step.2.

In [1]:
mkdir -p ~/SG-ONT-2021/RESULTS/BLOBTOOLS
cd ~/SG-ONT-2021/RESULTS/BLOBTOOLS

### Blastx using diamond 

Run diamond in blastx mode using assembled contigs vs a pre-formatted diamond bacteria database (protein)

In [2]:
#prepare assembly file
ASSEMBLY="/home/jovyan/SG-ONT-2021/RESULTS/FLYE_RACON_MEDAKA/MEDAKA_CONSENSUS/consensus.fasta"
#symbolic link from last assembly in the current repertory BLOBTOOLs
ln -s ${ASSEMBLY} ${CLONE}_FLYE_RACONx2_MEDAKA.fasta
# overwrite ASSEMBLY variable
ASSEMBLY=${CLONE}_FLYE_RACONx2_MEDAKA.fasta

In [7]:
conda activate diamond
time diamond blastx --query ${ASSEMBLY}  --db ~/SG-ONT-2021/DATA/testBacteria.dmnd  --outfmt 6 qseqid staxids bitscore qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore --sensitive  --max-target-seqs 1  --evalue 1e-25  --threads 4  --out diamond.csv
conda deactivate

(diamond) diamond v2.0.11.149 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 4
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: 
#Target sequences to report alignments for: 1
Opening the database...  [0.099s]
Database: /home/jovyan/SG-ONT-2021/DATA/testBacteria.dmnd (type: Diamond database, sequences: 271272, letters: 174523808)
Block size = 2000000000
Opening the input file...  [0.001s]
Opening the output file...  [0s]
Loading query sequences...  [0.026s]
Masking queries...  [0.124s]
Algorithm: Double-indexed
Building query histograms...  [0.176s]
Allocating buffers...  [0s]
Loading reference sequences...  [0.635s]
Masking reference...  [3.046s]
Initializing temporary storage...  [0.001s]
Building reference histograms...  [4.793s]
Allocating buffers...  [0s]
Proce

: 1

### Run blobtools 

Specifically to Blobtools: * nodes and names from ncbi taxdump database can be download from [here](https://github.com/DRL/blobtools#download-ncbi-taxdump-and-create-nodesdb).

In this training nodes and names are available on the DATA repertory

In [8]:
BAM="/home/jovyan/SG-ONT-2021/RESULTS/REMAPPING/${ASSEMBLY/.fasta/_ONT_minimap2.bam}"
DIAMONDX="/home/jovyan/SG-ONT-2021/RESULTS/BLOBTOOLS/diamond.csv"
BLOB_NODES="/home/jovyan/SG-ONT-2021/DATA/blobtools/nodes.dmp"
BLOB_NAMES="/home/jovyan/SG-ONT-2021/DATA/blobtools/names.dmp"

(base) (base) (base) (base) 

: 1

In [13]:
conda activate blobtools
#blobtools create -i "$ASSEMBLY" -b "$BAM" -t "${DIAMONDX}" -o "${ASSEMBLY/.fasta/_blobtools}" --db "$BLOB_DB"
#blobtools blobplot -i "${ASSEMBLY/.fasta/_blobtools}.blobDB.json" -o "${ASSEMBLY/.fasta/_blobtools}"
blobtools create -i ${ASSEMBLY} -b ${BAM} -t ${DIAMONDX} -o quality --names ${BLOB_NAMES} --nodes ${BLOB_NODES} --db nodesDB.txt
#blobtools plot -i "${ASSEMBLY/.fasta/_blobtools}.blobDB.json" -o "${ASSEMBLY/.fasta/_blobtools}"


(blobtools) (blobtools) (blobtools) [+] Parsing FASTA - _FLYE_RACONx2_MEDAKA.fasta
[+] Store nodesDB in nodesDB.txt
[%] : 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.29M/2.29M [00:04<00:00, 552kit/s]
[+] Parsing tax0 - /home/jovyan/SG-ONT-2021/RESULTS/BLOBTOOLS/diamond.csv
[+] Computing taxonomy using taxrule(s) bestsum
[%] : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3.00/3.00 [00:00<00:00, 27.1kit/s]
[+] Parsing bam0 - /home/jovyan/SG-ONT-2021/RESULTS/REMAPPING/_FLYE_RACONx2_MEDAKA_ONT_minimap2.bam
[E::hts_open_format] Failed to open file /home/jovyan/SG-ONT-2021/RESULTS/REMAPPING/_FLYE_RACONx2_MEDAKA_ONT_minimap2.bam
Traceback (most recent call last):
  File "/opt/conda/envs/blobtools/bin/blobtools", line 10, in <module>
    sys.exit(main())
  File "/op

: 1

In [ ]:
blobtools view -i quality.blobDB.json --cov -o output;
blobtools plot -i quality.blobDB.json;

In [ ]:
conda deactivate

#### Is there contamination in the assembly?

#### Do any assemblies show strange clustering?

#### Why might coverage vary across contigs within an assembly?

The Blobplots all indicate a single cluster. Some contigs show fairly high coverage in comparison to the rest of the genome which could be repetitive elements in the genome.

# 5. Comparative Alignment

Comparative alignment is a useful tool to see how assemblies compare to each other. This can be useful to compare assemblies to a reference, or to see if assemblies have large structural differences.



Verify assembled genome by dgenies : http://dgenies.toulouse.inra.fr/

Since you have a reference genome available, an alternative option to de novo assembly is to assemble using a reference genome - by mapping of the sequence reads

Prepare data to download into dgenies : 

 * Reference.fasta ` ~/SG-ONT-2021/DATA/CloneX/reference.fasta `
 
 * ONT assembly in Clone20 (Flye+Raconx2+Medaka) ` ~/SG-ONT-2021/DATA/DGENIES/FRx2M_Clone20.fasta" ` 
 
 * ABYSS assembly generated with illumina reads  ` ~/SG-ONT-2021/DATA/DGENIES/Abyss_Clone20.fasta ` FRANCOIS!
 

#### What is the mean difference between illumina and ONT assembly for Clone20?